In [11]:
from src.vdb import get_weaviate_client
from src.vdb import EventRetriever
from src.vdb import COLLECTION_NAME

In [12]:
client = get_weaviate_client()
collection = client.collections.get(COLLECTION_NAME)

In [15]:
import weaviate.classes as wvc
ob = collection.query.near_text(
                query='москва',
                limit=445 * 2,  # Берем больше результатов для последующей фильтрации
                return_metadata=wvc.query.MetadataQuery(distance=True),
            )

In [29]:
set([own.properties['owner'] for own in ob.objects])

{None, 'all'}

In [28]:
import numpy as np
np.unique(np.array([own.properties['owner'] for own in ob.objects]))

TypeError: '<' not supported between instances of 'NoneType' and 'str'

In [ ]:
Object(uuid=_WeaviateUUIDInt('e3f77a20-e770-5da8-9954-224a7984d413'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=0.3695290684700012, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'description': 'Как проходило строительство небоскрёбов «Москва-Сити», что скрывается за\xa0их\xa0стеклянными фасадами и\xa0люди каких профессий работают над обликом города? На\xa0смотровой площадке PANORAMA360\xa0в увлекательной форме школьники познакомятся с\xa0интересными фактами о\xa0создании современной городской среды.', 'tags': ['all', 'kids', 'msk', 'интересное', 'детям', '6+', 'экскурсии', 'экскурсии для детей', 'развлечения', 'красивый вид', 'школьники', '6+'], 'uuid': 'e3f77a20-e770-5da8-9954-224a7984d413', 'date': None, 'source': 'kudago', 'url': 'https://kudago.com/msk/event/detyam-pokoriteli-vershin/', 'owner': 'all', 'country': 'Россия', 'title': 'детская интерактивная экскурсия «Покорители вершин» на смотровой площадке PANORAMA360', 'location': 'смотровая площадка PANORAMA360 в\xa0ММДЦ «Москва-Сити», наб. Пресненская, д. 12,  ММДЦ «Москва-Сити», башня «Федерация. Восток», этаж 89, м. Выставочная, Деловой центр'}, references=None, vector={}, collection='Events')

In [23]:
for obj in ob.objects:
    if obj.properties['owner'] == 'user_679913517':
        print(obj.title)
        print(obj.description)
        print(obj.location)
        print(obj.date)
        print(obj.url)
        print()

In [3]:
retriever = EventRetriever()

In [8]:
events = retriever.retrieve("москва", owner="user_679913517")
for event in events[:4]:
    print('=====')
    print(event.title)
    print(event.description)


In [10]:
client = get_weaviate_client()
# Получить все данные из коллекции (современный Weaviate API)
collection = client.collections.get(COLLECTION_NAME)


dict_keys([])

In [12]:
# Получить объекты с метаданными (UUID, время создания, вектор и т.д.)
client = get_weaviate_client()

# Проверяем существование коллекции
if COLLECTION_NAME in client.collections.list_all():
    collection = client.collections.get(COLLECTION_NAME)
    
    # Получить объекты с метаданными
    import weaviate.classes as wvc
    result = collection.query.fetch_objects(
        limit=100,
        return_metadata=wvc.query.MetadataQuery(
            uuid=True,
            creation_time=True,
            distance=True,
        )
    )
    
    # Вывести с метаданными
    print(f"📊 Найдено объектов: {len(result.objects)}\n")
    for i, obj in enumerate(result.objects, 1):
        print(f"{i}. UUID: {obj.uuid}")
        if hasattr(obj.metadata, 'creation_time') and obj.metadata.creation_time:
            print(f"   Создано: {obj.metadata.creation_time}")
        print(f"   Название: {obj.properties.get('title', 'N/A')}")
        print(f"   Теги: {obj.properties.get('tags', [])}")
        print("-" * 70)
else:
    print(f"❌ Коллекция '{COLLECTION_NAME}' не существует")
    print(f"💡 Создайте её командой: python src/vdb/scripts/add_events.py")

client.close()

WeaviateQueryError: Query call with protocol GRPC search failed with message could not find class Events in schema.

In [3]:
client = get_weaviate_client()
client.collections.delete('events')

In [11]:
# 🗑️ Удалить коллекцию
# ВНИМАНИЕ: Это безвозвратно удалит коллекцию и все данные!

client = get_weaviate_client()

# Проверяем существование коллекции
if COLLECTION_NAME in client.collections.list_all():
    print(f"⚠️  Коллекция '{COLLECTION_NAME}' найдена")
    
    # Получаем количество объектов перед удалением
    collection = client.collections.get(COLLECTION_NAME)
    result = collection.query.fetch_objects(limit=1)
    # Примерное количество (для больших коллекций нужен отдельный запрос)
    
    confirm = input(f"Вы уверены, что хотите удалить коллекцию '{COLLECTION_NAME}'? (yes/no): ")
    
    if confirm.lower() == 'yes':
        client.collections.delete(COLLECTION_NAME)
        print(f"✅ Коллекция '{COLLECTION_NAME}' успешно удалена!")
    else:
        print("❌ Удаление отменено")
else:
    print(f"ℹ️  Коллекция '{COLLECTION_NAME}' не существует")
    print(f"📋 Доступные коллекции: {client.collections.list_all()}")

client.close()


ℹ️  Коллекция 'Events' не существует
📋 Доступные коллекции: {}


In [4]:
# 📥 Тестирование парсера KudaGo
from src.data_parsers.kudago_parser import parse_kudago_json, parse_all_kudago_files

# Парсим один файл
json_path = "/Users/andreisuhov/Desktop/llm_journer/data/raw_data/real_events_data/events_pydantic_spb_20251212_131151.json"
events = parse_kudago_json(json_path, owner="all")

print(f"✅ Загружено событий: {len(events)}\n")

# Показываем первые 3 события
for i, event in enumerate(events[:3], 1):
    print(f"{i}. {event.title}")
    print(f"   Описание: {event.description[:100]}..." if len(event.description) > 100 else f"   Описание: {event.description}")
    print(f"   Местоположение: {event.location}")
    print(f"   Дата: {event.date}")
    print(f"   Теги: {', '.join(event.tags[:5])}...")
    print(f"   URL: {event.url}")
    print()

✅ Загружено событий: 518

1. Как раскрыть тайны гения: концерты музыки Баха от Amadeus Concerts
   Описание: Иоганн Себастьян Бах имеет такое же значение для мировой музыки, как, скажем, Пушкин для русской лит...
   Местоположение: Петрикирхе — кафедральный собор Святых Петра и Павла, просп. Невский, д. 22–24, м. Невский Проспект
   Дата: None
   Теги: all, concert, spb, концерты...
   URL: https://kudago.com/spb/event/kontsert-kontsertyi-muzyiki-baha-ot-amadeus-concerts/

2. музыкальный проект #В_СВЕЧАХ в особняках и дворцах Петербурга
   Описание: Новый проект от «Русских Музыкальных Сезонов» — это магия музыки, помноженная на магию свечей и крас...
   Местоположение: None
   Дата: 2023-07-06 20:00 - 2023-08-30 20:00
   Теги: all, concert, spb, 6+, лучшие события 8 марта...
   URL: https://kudago.com/spb/event/kontsert-vsvechahh/

3. концерты «Музыка в Оранжерее» проекта «Русские Музыкальные сезоны»
   Описание: Любовь проявляется в самых повседневных вещах. Эта идея и легла в основу

In [5]:
events[0]

Event(title='Как раскрыть тайны гения: концерты музыки Баха от Amadeus Concerts', owner='all', description='Иоганн Себастьян Бах имеет такое же значение для мировой музыки, как, скажем, Пушкин для русской литературы. Однако о личности великого немецкого кантора известно крайне мало. Почему так случилось, и можно ли разгадать его тайны, расскажем в этом материале.', tags=['all', 'concert', 'spb', 'концерты'], source='kudago', country='Россия', location='Петрикирхе — кафедральный собор Святых Петра и Павла, просп. Невский, д. 22–24, м. Невский Проспект', date=None, url='https://kudago.com/spb/event/kontsert-kontsertyi-muzyiki-baha-ot-amadeus-concerts/')

In [4]:
# 📂 Парсинг всех файлов из директории
from src.data_parsers.kudago_parser import parse_all_kudago_files

data_dir = "/Users/andreisuhov/Desktop/llm_journer/data/raw_data/real_events_data"
all_events = parse_all_kudago_files(data_dir, owner="user123")

print(f"\n📊 Статистика:")
print(f"Всего событий: {len(all_events)}")

# Подсчет по категориям
categories = {}
for event in all_events:
    for tag in event.tags:
        if tag in ['concert', 'exhibition', 'show', 'festival']:
            categories[tag] = categories.get(tag, 0) + 1

print(f"\nПо категориям:")
for cat, count in sorted(categories.items(), key=lambda x: x[1], reverse=True):
    print(f"  {cat}: {count}")

# Показываем несколько случайных событий
import random
print(f"\n🎲 Несколько случайных событий:")
for event in random.sample(all_events, min(5, len(all_events))):
    print(f"• {event.title}")
    print(f"  {event.location or 'Местоположение не указано'}")
    print()

[]

In [12]:

from src.vdb import create_collection_if_not_exists

create_collection_if_not_exists()

ℹ️  Создаю коллекцию 'Events'
✅ Коллекция 'Events' создана


In [ ]:
#654

In [5]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ResourceWarning)


from src.launch_pipeline import launch_pipeline

launch_pipeline()

Ожидание подключения к Weaviate по адресу: http://localhost:8080
Таймаут: 60 секунд
✓ Weaviate подключен успешно за 0.34 секунд
ℹ️  Коллекция 'Events' уже существует
   Количество записей в коллекции: 664
   Для пересоздания с новой конфигурацией используйте: python /Users/andreisuhov/Desktop/llm_journer/src/vdb/utils/add_events.py --recreate
Parsing file: /Users/andreisuhov/Desktop/llm_journer/data/raw_data/real_events_data/events_pydantic_msk_20251212_130919.json
Loaded 570 events
Loading events to Weaviate
📤 Загрузка 570 событий в Weaviate...

Загрузка завершена!
   Успешно: 0
   Ошибок: 0
   Пропущено: 570
Parsing file: /Users/andreisuhov/Desktop/llm_journer/data/raw_data/real_events_data/events_pydantic_spb_20251212_131151.json
Loaded 518 events
Loading events to Weaviate
📤 Загрузка 518 событий в Weaviate...

Загрузка завершена!
   Успешно: 0
   Ошибок: 0
   Пропущено: 518


In [1]:

from pathlib import Path

from src.data_parsers.kudago_parser import parse_all_kudago_files
from src.vdb.utils.test_connection import wait_for_weaviate
from src.vdb.scripts.add_events import create_collection_if_not_exists
from src.vdb.scripts.load_kudago_events import load_events_to_weaviate

from src.utils.paths import DATA    



file_paths = [
    Path(DATA / "raw_data/real_events_data/events_pydantic_msk_20251212_130919.json"),
    Path(DATA / "raw_data/real_events_data/events_pydantic_spb_20251212_131151.json"),
]




wait_for_weaviate()
create_collection_if_not_exists()
for file_path in file_paths:
    print(f"Parsing file: {file_path}")
    events = parse_all_kudago_files(file_path, owner="all")
    print(f"Loaded {len(events)} events")
    print(f"Loading events to Weaviate")
    load_events_to_weaviate(events, batch_size=100, verbose=True)


Ожидание подключения к Weaviate по адресу: http://localhost:8080
Таймаут: 60 секунд
✓ Weaviate подключен успешно за 0.30 секунд
ℹ️  Коллекция 'Events' уже существует
   Количество записей в коллекции: 0
   Для пересоздания с новой конфигурацией используйте: python /Users/andreisuhov/Desktop/llm_journer/src/vdb/scripts/add_events.py --recreate
Parsing file: /Users/andreisuhov/Desktop/llm_journer/data/raw_data/real_events_data/events_pydantic_msk_20251212_130919.json
Найдено 0 файлов с событиями

Всего событий загружено: 0
Loaded 0 events
Loading events to Weaviate


/Users/andreisuhov/miniconda3/envs/env/lib/python3.11/site-packages/weaviate/warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/Users/andreisuhov/miniconda3/envs/env/lib/python3.11/json/decoder.py:353: ResourceWarning: unclosed <socket.socket fd=75, family=30, type=1, proto=6, laddr=('::1', 49640, 0, 0), raddr=('::1', 8080, 0, 0)>
  obj, end = self.scan_once(s, idx)


📤 Загрузка 0 событий в Weaviate...

✅ Загрузка завершена!
   Успешно: 0
   Ошибок: 0
   Пропущено: 0
Parsing file: /Users/andreisuhov/Desktop/llm_journer/data/raw_data/real_events_data/events_pydantic_spb_20251212_131151.json
Найдено 0 файлов с событиями

Всего событий загружено: 0
Loaded 0 events
Loading events to Weaviate
📤 Загрузка 0 событий в Weaviate...

✅ Загрузка завершена!
   Успешно: 0
   Ошибок: 0
   Пропущено: 0


In [ ]:
parse_all_kudago_files

In [1]:
from src.vdb.rag.self_rag_graph import run_self_rag


/Users/andreisuhov/miniconda3/envs/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.utils.journey_llm import JourneyLLM
from src.vdb import EventRetriever

llm = JourneyLLM()
retriever = EventRetriever()
query = "хочу на выставку, а потом поесть"

res = run_self_rag(query, retriever=retriever, llm=llm)

In [3]:
print(res)

events=[Event(title='серия кинопоказов «Международный день кино»', owner='all', description='Золотая коллекция мирового кинематографа оживёт на большом экране всего за один день. В залах «Иллюзиона» покажут более двадцати фильмов — от американских нуаров до отреставрированной советской классики. Это редкая возможность увидеть шедевры в плёночном или цифровом формате из фондов Госфильмофонда.', tags=['all', 'cinema', 'msk', 'интересное', '12+', 'культура и искусство', 'главное в городе', 'кинопоказы', 'в свободное время', '12+'], source='kudago', country='Россия', location='кинотеатр «Иллюзион», наб. Котельническая, д. 1/15, м. Китай-город, Марксистская, Таганская', date='2025-12-28 10:00', url='https://kudago.com/msk/event/seriya-kinopokazov-mezhdunarodnyij-den-kino-oeftes/', uuid='17319c66-02ee-551e-8b21-18c6e6443fcd'), Event(title='живые знакомства в формате GOKON', owner='all', description='Компания GOKON приглашает на групповые свидания в необычном формате. Миссия проекта — помочь 

In [4]:
from src.planner_agent.graph import PlanningGraph

In [5]:
res

InputData(events=[Event(title='серия кинопоказов «Международный день кино»', owner='all', description='Золотая коллекция мирового кинематографа оживёт на большом экране всего за один день. В залах «Иллюзиона» покажут более двадцати фильмов — от американских нуаров до отреставрированной советской классики. Это редкая возможность увидеть шедевры в плёночном или цифровом формате из фондов Госфильмофонда.', tags=['all', 'cinema', 'msk', 'интересное', '12+', 'культура и искусство', 'главное в городе', 'кинопоказы', 'в свободное время', '12+'], source='kudago', country='Россия', location='кинотеатр «Иллюзион», наб. Котельническая, д. 1/15, м. Китай-город, Марксистская, Таганская', date='2025-12-28 10:00', url='https://kudago.com/msk/event/seriya-kinopokazov-mezhdunarodnyij-den-kino-oeftes/', uuid='17319c66-02ee-551e-8b21-18c6e6443fcd'), Event(title='живые знакомства в формате GOKON', owner='all', description='Компания GOKON приглашает на групповые свидания в необычном формате. Миссия проекта

In [6]:
from src.planner_agent.graph import PlanningGraph

llm = JourneyLLM()
graph = PlanningGraph(llm)

output = graph.run(res)
print(output)


🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
НАЧАЛО РАБОТЫ ГРАФА ПЛАНИРОВАНИЯ
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
Событий для планирования: 15
Максимальное количество итераций: 1

▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶
УЗЕЛ: planner_reasoning
▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶

🤔 ПЛАНИРОВЩИК: Начинаю анализ и рассуждения...
Анализирую 15 событий
Отправляю запрос к LLM для анализа...
💡 LLM может использовать инструменты для получения дополнительной информации
✅ Рассуждения получены:
   - Соображений: 3
   - Проблем выявлено: 3
   - Стратегия: Сначала определить, какие выставки доступны для посещения, затем найти места для еды рядом с ними и ...

▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶
УЗЕЛ: planner_create
▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶▶

📋 ПЛАНИРОВЩИК: Создаю план похода...
Использую предыдущие рассуждения
Отправляю запрос к LLM для создания плана...
💡 LLM может использовать инструменты для получения информации о маршрутах и погоде
✅ План создан:
   - Событий в плане: 4
   - Общая продолжительность: 300 минут
   - Время в пут

In [7]:
print(output.final_text)

1. 10:00:00+00:00–12:00:00+00:00 — Постоянная экспозиция в Домике Петра Первого — Домик Петра I, наб. Петровская, д. 6, м. Горьковская (car, 30 мин)
   Посещение выставки в Домике Петра Первого.
2. 12:00:00+00:00–13:00:00+00:00 — Поесть — Ресторан рядом с Домиком Петра Первого (walking, 5 мин)
   Поесть в ресторане рядом с выставкой.
3. 13:00:00+00:00–15:00:00+00:00 — Коллекция Эрмитажа «Тульский художественный металл XVIII—XIX веков» — Эрмитаж (Зимний дворец, Главный музейный комплекс), пл. Дворцовая, д. 2, м. Адмиралтейская, Гостиный двор (car, 20 мин)
   Посещение коллекции Эрмитажа.
4. 15:00:00+00:00–17:00:00+00:00 — Живые знакомства в формате GOKON — Адрес не указан (bus, 30 мин)
   Посещение мероприятия GOKON.


In [9]:
output

OutputResult(final_plan=Plan(items=[], total_duration_minutes=0, total_travel_time_minutes=0, summary='Необходима дополнительная информация о выставке и ресторане для составления плана похода.', included_events=[], excluded_events=['Отсутствие информации о выставке и ресторане.']), reasoning=Reasoning(analysis='Пользователь хочет посетить выставку и поесть после этого. Однако, конкретные детали о времени, месте и продолжительности событий отсутствуют, что затрудняет планирование. Необходимо уточнить, где проходит выставка и где пользователь хочет поесть, чтобы определить оптимальный маршрут и транспорт.', considerations=['Уточнить местоположение выставки и ресторана.', 'Определить предпочтения пользователя по времени посещения выставки и обеда.', 'Учитывать возможные ограничения по времени и бюджету.'], challenges=['Отсутствие конкретной информации о времени и месте событий.', 'Неизвестные предпочтения пользователя по транспорту и бюджету.'], strategy='Сначала необходимо выяснить детал

In [ ]:
output.final_plan.items

[PlanItem(event_name='Выставка', event_address='Место проведения выставки', start_time=datetime.time(10, 0, tzinfo=TzInfo(UTC)), end_time=datetime.time(12, 0, tzinfo=TzInfo(UTC)), duration_minutes=120, transport_mode='car', travel_time_minutes=20, notes='Посетить выставку.'),
 PlanItem(event_name='Ресторан', event_address='Ближайший ресторан', start_time=datetime.time(12, 30, tzinfo=TzInfo(UTC)), end_time=datetime.time(13, 30, tzinfo=TzInfo(UTC)), duration_minutes=60, transport_mode='walking', travel_time_minutes=5, notes='Пообедать в ресторане.')]